<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/ndl_ocr_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NDLOCRの実行例 Ver.2：Google Driveを用いた画像の入力と結果の保存

Google Drive上のフォルダを指定して、当該フォルダに含まれる複数画像に対してOCR処理を実行し、指定したフォルダに認識結果を出力します。

**本ノートブック（Ver.2）では、PDFファイルやIIIFマニフェストファイルの入力をサポートしています。また、実行後に認識結果をマージしたテキストファイルをダウンロードすることができます。**

Ver.1は以下です。

https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/ndl_ocr_folder.ipynb

参考：https://github.com/blue0620/NDLOCR-GoogleColabVersion/blob/main/NDLOCR_googlecolabversion.ipynb





## 使用方法

- 「1.初期セットアップ」を実行してください。初回のみ実行が必要です。

- 「2. 設定」を変更してください。

## 0.GPUを使用するか否か

GPUを使用しない場合には、チェックを外して実行ボタンを押してください。

In [ ]:
#@title 全体の設定
use_gpu = True #@param {type:"boolean"}

## 1.初期セットアップ

時間がかかります。初回のみ実行が必要です。

In [ ]:
# @title

%cd "/content/"

# 6. google driveのマウント
from google.colab import drive
drive.mount('/content/drive/')

from IPython.display import clear_output 

# 1. NDLOCRのリポジトリをcloneする(--recursiveを忘れずに！)
!rm -rf ndlocr_cli
!git clone -q --recursive https://github.com/nakamura196/ndlocr_cli
clear_output()

# 2. 必要なパッケージをインストールする
PROJECT_DIR="/content/ndlocr_cli"
!pip install -q -r {PROJECT_DIR}/requirements.txt
!pip install -q torch==1.8.1+cu111 torchvision==0.9.1+cu111 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
clear_output()
!pip install -q mmcv-full==1.4.0 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.8.0/index.html
##numpyのバージョン問題でcolabでは動かなかったのでアップデートする(参考:https://stackoverflow.com/questions/66060487/valueerror-numpy-ndarray-size-changed-may-indicate-binary-incompatibility-exp)
!pip install -q --upgrade numpy
clear_output()

%cd {PROJECT_DIR}/src/ndl_layout/mmdetection
!python setup.py bdist_wheel
clear_output()
!pip install -q dist/*.whl
clear_output()

# # 4. OCRに必要な学習済みモデルをダウンロードする

%cd {PROJECT_DIR}
!wget -q https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/mojilist_NDL.txt -P ./src/text_recognition/models
!wget -q https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/ndlenfixed64-mj0-synth1.pth -P ./src/text_recognition/models
!wget -q https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/ndl_layout_config.py -P ./src/ndl_layout/models
!wget -q https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/epoch_140_all_eql_bt.pth -P ./src/ndl_layout/models
!wget -q https://lab.ndl.go.jp/dataset/ndlocr/separate_pages_ssd/weights.hdf5 -P ./src/separate_pages_ssd/ssd_tools
clear_output()

# 5. 環境変数を追加する
import os
os.environ["PYTHONPATH"]=os.environ["PYTHONPATH"]+":"+f"{PROJECT_DIR}/src/text_recognition/deep-text-recognition-benchmark"

clear_output()

# PDF関連
!apt-get install poppler-utils
!pip install -q pdf2image

clear_output()

In [ ]:
#@title 独自関数のロード

# %cd {PROJECT_DIR}

from pathlib import Path
import glob
from tqdm import tqdm
from pdf2image import convert_from_path
import os
import datetime
import pytz
from google.colab import files
import requests
import json
from urllib import request

def getProcessParam(process):
  p = "0..3"
  if process == "傾き補正,レイアウト抽出,文字認識(OCR)":
    p = "1..3"
  elif process == "レイアウト抽出,文字認識(OCR)":
    p = "2..3"
  elif process == "文字認識(OCR)":
    p = "3"
  return p

def createParentDir(path):
  basename = os.path.basename(path)
  parent_path = os.path.dirname(path)
  os.makedirs(parent_path, exist_ok=True)

def listFiles(input_dir, exts):
  paths = []
  ext_list = exts.split(",")
  for ext in ext_list:
    files = glob.glob("{}/**/*.{}".format(input_dir, ext), recursive=True)
    paths.extend(files)
  return sorted(paths)

def prepareImgDir(file_id):
  print("### 入力画像フォルダの作成 ###")

  run_id = datetime.datetime.now(pytz.timezone('Asia/Tokyo')).strftime('%Y%m%d%H%M%S')
  tmp_dir = "/tmp/" + run_id + "/" + file_id

  '''
  if os.path.exists(tmp_dir):
    !rm -rf $tmp_dir
  '''

  # img_dir
  tmp_img_dir = tmp_dir + "/img"
  os.makedirs(tmp_img_dir, exist_ok=True)

  return tmp_dir, tmp_img_dir

# 画像のダウンロード
def downloadImages(url, output_dir):
  print("### IIIFマニフェストを用いた画像のダウンロード ###")
  df = requests.get(url).json()
  canvases = df["sequences"][0]["canvases"]
  if process_size > 0:
    canvases = canvases[0:process_size]
  for i in tqdm(range(len(canvases))):
    res = canvases[i]["images"][0]["resource"]
    index = str(i+1).zfill(4)
    path = "{}/{}.jpg".format(output_dir, index)
    os.makedirs(os.path.dirname(path), exist_ok=True)

    '''
    if "service" in res:
      info_url = res["service"]["@id"]
      info = requests.get(info_url + "/info.json").json()

      json_path = "{}/json/{}.json".format(etc_dir, index)
      os.makedirs(os.path.dirname(json_path), exist_ok=True)
      json.dump(info, open(json_path, "w"))

      image_width = info["width"]
      image_height = info["height"]

      # 短辺が1600になるように
      size = ",1600"

      if image_width < image_height:
        size = "1600,"
      url = "{}/full/{}/0/default.jpg".format(info_url, size)
      
    else:
    '''
    url = canvases[i]["images"][0]["resource"]["@id"]
    request.urlretrieve(url, path)

def createImagesFromPDF(pdf_path, img_dir, filename):
  print("### PDFファイルから画像ファイルの作成 ###")
  pdf_path = Path(pdf_path)
  img_path=Path(img_dir)
  convert_from_path(pdf_path, output_folder=img_path,fmt='jpeg', output_file=pdf_path.stem,dpi=100)

def execByLocalPdf(file, process):
  filename = getFileIdAndName(file)
  tmp_dir, tmp_img_dir = prepareImgDir(filename)
  createImagesFromPDF(file, tmp_img_dir, filename)
  main(tmp_dir, output_dir, process)

def getFileIdAndName(path):
  print("### ファイルIDと名前の取得 ###")
  return path.split("/")[-1].split(".")[0]

def main(input_dir, output_dir, process):
  print("### 推論の実行 ###")

  output_dir_by_param = createOutputDirIfExist(output_dir)

  p = getProcessParam(process)

  # CPUのみを使用する場合
  c = "config.yml"
  try:
    if not use_gpu:
      c = "config_cpu.yml"
  except:
    pass

  !python main.py infer -c $c -s s $input_dir $output_dir_by_param -x -i -p $p
  clear_output()

  mergeTxtAndDownload(output_dir_by_param)

# 出力フォルダの設定

def createOutputDirIfExist(output_dir):
  if os.path.exists(output_dir): 
    run_id = datetime.datetime.now(pytz.timezone('Asia/Tokyo')).strftime('%Y%m%d%H%M%S')
    output_dir = output_dir + "_" + run_id
    print("!!! 指定された出力フォルダが既に存在するため、 {} に出力します。 !!!".format(output_dir))

  # 親フォルダの作成
  createParentDir(output_dir)

  return output_dir

# txtファイルにまとめる
def mergeTxtAndDownload(output_dir):
  tmp_dirs = glob.glob(output_dir + "/*/txt")
  txt_dir = tmp_dirs[0]
  
  file_id = txt_dir.split("/")[-2]
  output_id_dir = output_dir + "/" + file_id

  txt_files = glob.glob(txt_dir + "/*.txt")
  txt_files = sorted(txt_files)
  merged_txt_file_path = output_id_dir + "/" + file_id + '.txt'
  merged_txt_file = open(merged_txt_file_path, 'a')
  for each_txt_file_path in txt_files:
    each_txt_file = open(each_txt_file_path, 'r')
    data = each_txt_file.read()
    each_txt_file.close()
    merged_txt_file.write(data+"\n")
  merged_txt_file.close()

  files.download(merged_txt_file_path)

## 2.設定

以下、入力方式によって適切なものを選んでください。

- 2.1 既にダウンロード済みの画像ファイルを対象にする場合（Sigle input dir mode）
- 2.2 既にダウンロード済みの画像ファイルを対象にする場合（Image file mode: 単体の画像ファイルを入力として与える場合）
- 2.3 既にダウンロード済みのPDFファイルを対象にする場合
- 2.4 PDFファイルのURLを指定する場合
- 2.5 IIIFマニフェストファイルのURLを指定する場合

### 2.1 既にダウンロード済みの画像ファイルを対象にする場合（Sigle input dir mode）

- input_dir: 入力するフォルダのパス
  - 指定したフォルダの下にimgフォルダを用意し、その中に画像を格納してください。
- output_dir: 出力するフォルダへのパス

In [ ]:
#@title 設定

input_dir = "/content/drive/MyDrive/ndl_ocr/input" #@param {type:"string"}
output_dir = "/content/drive/MyDrive/ndl_ocr/output" #@param {type:"string"}
process = "\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]

output_dir = str(Path(output_dir))
input_dir = str(Path(input_dir))

In [ ]:
#@title 実行
main(input_dir, output_dir, process)

### 2.2 既にダウンロード済みの画像ファイルを対象にする場合（Image file mode: 単体の画像ファイルを入力として与える場合）

以下のノートブック（Ver.1）をご利用ください。

https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/ndl_ocr_folder.ipynb

### 2.3 既にダウンロード済みのPDFファイルを対象にする場合

- input_file: PDFファイルのパス
- output_dir: 出力するフォルダへのパス

In [ ]:
#@title　設定

input_file = "/content/drive/MyDrive/ndl_ocr/input/pdfs/tygz-001.pdf" #@param {type:"string"}
# input_dir = "/content/drive/MyDrive/ndl_ocr/input/pdfs"
output_dir = "/content/drive/MyDrive/ndl_ocr/output/pdf_local" #@param {type:"string"}
process = "\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]

output_dir = str(Path(output_dir))
# input_dir = str(Path(input_dir))

In [ ]:
#@title 実行
execByLocalPdf(input_file, process)

# 複数ファイルの場合（TODO）
'''
target_files = glob.glob(input_dir+"/**/*.pdf", recursive=True)
for file in target_files:
  execByLocalPdf(file)
'''

pass

### 2.4 PDFファイルのURLを指定する場合

- url: PDFファイルのURL
- output_dir: 出力するフォルダへのパス

In [ ]:
#@title 設定

url = "https://dglb01.ninjal.ac.jp/ninjaldl/toyogakuge/001/PDF/tygz-001.pdf" #@param {type:"string"}
output_dir = "/content/drive/MyDrive/ndl_ocr/output/pdf_url" #@param {type:"string"}
process = "\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]

output_dir = str(Path(output_dir))
# input_dir = str(Path(input_dir))

In [ ]:
#@title　実行

filename = getFileIdAndName(url)
tmp_dir, tmp_img_dir = prepareImgDir(filename)

print("### PDFファイルのダウンロード ###")
tmp_pdf_path = tmp_dir + "/" + filename + ".pdf"
!curl $url -o $tmp_pdf_path

createImagesFromPDF(tmp_pdf_path, tmp_img_dir, filename)

main(tmp_dir, output_dir, process)

### 2.5 IIIFマニフェストファイルのURLを指定する場合

- url: IIIFマニフェストファイルのURL
- output_dir: 出力するフォルダへのパス
- process_size: 処理するcanvas数。-1を指定するとすべてのcanvasを処理対象とする。負荷軽減のため、初期値は5に設定しています。

In [ ]:
#@title 設定

url = "https://www.dl.ndl.go.jp/api/iiif/3437686/manifest.json" #@param {type:"string"}
output_dir = "/content/drive/MyDrive/ndl_ocr/output/iiif" #@param {type:"string"}
process = "\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]

process_size = 5  #@param {type:"number"}

output_dir = str(Path(output_dir))

In [ ]:
#@title 実行
filename = getFileIdAndName(url)
tmp_dir, tmp_img_dir = prepareImgDir(filename)
downloadImages(url, tmp_img_dir)
main(tmp_dir, output_dir, process)